`libics.driver.camera`
`libics v1.0`

# VRmagic VRmC-X

Product homepage: [Imaging USB Platform](https://www.vrmagic-imaging.com/service/downloads/usb-platform-downloads)

In [ ]:
from libics.driver import camera

In [ ]:
import IPython
from io import BytesIO
import matplotlib as mpl
import numpy as np
import PIL

def display_array(ar, d, cmap="viridis", vmin=None, vmax=None, height=512):
    """
    ar : `np.ndarray(2)`
        2D array to be displayed.
    d : `dict`
        Pass a new empty dictionary, i.e. `dict()`.
        This is required for the algorithm.
    """
    if "display_handle" not in d:
        d["display_handle"] = None
    
    if vmin is None:
        vmin = np.min(ar)
    if vmax is None:
        vmax = np.max(ar)
    ar_norm = (ar - vmin) / (vmax - vmin)
    im = getattr(mpl.cm, cmap)(ar_norm)
    im = (im * 255).astype(np.uint8)
    if im.shape[-1] == 4:
        im = im[:, :, :3]
    
    f = BytesIO()
    PIL.Image.fromarray(im).save(f, "jpeg")
    ratio = im.shape[1] / im.shape[0]
    h = height
    w = int(512 * ratio)
    display_obj = IPython.display.Image(data=f.getvalue(), width=w, height=h)
    if d["display_handle"] is None:
        d["display_handle"] = IPython.display.display(display_obj, display_id=True)
    else:
        d["display_handle"].update(display_obj)
    IPython.display.clear_output(wait=True)

## Device handle

In [ ]:
# Create communication interface
itf = camera.vrmagic.ItfVRmagic()
# Discover available device IDs
dev_ids = itf.discover()
dev_ids

In [ ]:
# Create device object
dev = camera.vrmagic.VRmagicVRmCX()
# Configure device
dev.configure(
    identifier=dev_ids[0], model=camera.VRMAGIC_VRMCX, interface=itf
)

## Usage

### Example 1 (basic usage)

In [ ]:
# Set up and connect to device
dev.setup()
dev.connect()
dev.is_connected()

In [ ]:
# Run device without callback (start acquisition loop)
dev.run(callback=None, blocking=False)
dev.is_running()

In [ ]:
# Grab single image
im = dev.grab()
# Show image
import matplotlib.pyplot as plt
plt.imshow(im, cmap="jet")
plt.colorbar()

In [ ]:
# Stop device (end acquisition loop)
dev.stop()
dev.is_running()

In [ ]:
# Close and shut down device
dev.close()
dev.shutdown()

### Example 2 (property settings)

In [ ]:
import matplotlib.pyplot as plt

with dev:
    print("Reading configuration")
    dev.p.read_all()
    print("  Image format: {:d}x{:d} px".format(dev.p.pixel_hrzt_count, dev.p.pixel_vert_count))
    print("  Pixel size: {:.1f}x{:.1f} µm".format(dev.p.pixel_hrzt_size * 1e6, dev.p.pixel_vert_size * 1e6))
    print("Setting exposure time")
    dev.write_exposure_time(10e-3)
    print("  Exposure time: {:.1f} ms".format(dev.p.exposure_time * 1e3))
    print("Taking image")
    dev.run()
    im = dev.grab()
    dev.stop()
    plt.imshow(im, vmin=0, vmax=255)
    plt.show()
    print("Setting exposure time")
    dev.write_exposure_time(300e-3)
    print("  Exposure time: {:.1f} ms".format(dev.p.exposure_time * 1e3))
    print("Taking image")
    dev.run()
    im = dev.grab()
    dev.stop()
    plt.imshow(im, vmin=0, vmax=255)
    plt.show()

### Example 3 (callback processing)

In [ ]:
dev.setup()
dev.connect()
dev.is_connected()

In [ ]:
dev.write_exposure_time(3e-3)

In [ ]:
_d = {}
dev.run(callback=lambda x: display_array(x, _d, cmap="jet", vmin=0, vmax=255))
dev.is_running()

In [ ]:
dev.stop()
dev.is_running()

In [ ]:
dev.close()
dev.shutdown()

## Raw Vendor API

In [ ]:
import ctypes as ct
import numpy as np
import matplotlib.pyplot as plt
import libics.driver.camera.vrmagic.vrmusbcam2 as vrm

In [ ]:
cam_id = 8514133

vrm.VRmUsbCamUpdateDeviceKeyList()

dev_count = vrm.VRmDWORD()
vrm.VRmUsbCamGetDeviceKeyListSize(ct.byref(dev_count))
dev_count = int(dev_count.value)

dev_key = None
dev_keys = [vrm.POINTER(vrm.VRmDeviceKey)() for _ in range(dev_count)]
[vrm.VRmUsbCamGetDeviceKeyListEntry(i, ct.byref(dev_key)) for i, dev_key in enumerate(dev_keys)]
for i, _dev_key in enumerate(dev_keys):
    print("Device {:d} ({:d}): {:s}".format(i, _dev_key.contents.m_serial, "busy" if _dev_key.contents.m_busy == 1 else "free"))
    if _dev_key.contents.m_serial == cam_id:
        dev_key = _dev_key
    else:
        vrm.VRmUsbCamFreeDeviceKey(_dev_key)

print("Model: {:s}".format(dev_key.contents.mp_product_str.data.decode('utf-8')))
dev_handle = vrm.VRmUsbCamDevice()
vrm.VRmUsbCamOpenDevice(dev_key, ct.byref(dev_handle))

####################

src_fmt = vrm.VRM_GRAY_8
vrm.VRmUsbCamSetSourceFormatIndex(dev_handle, src_fmt)


propid = vrm.VRmPropId()
propinfo = vrm.VRmPropInfo()
prop_attr_b = vrm.VRmPropAttribsB()
prop_b = vrm.VRmBOOL()
prop_attr_d = vrm.VRmPropAttribsD()
prop_d = vrm.c_double()
prop_attr_i = vrm.VRmPropAttribsI()
prop_i = ct.c_int()
prop_attr_f = vrm.VRmPropAttribsF()
prop_f = ct.c_float()
prop_attr_e = vrm.VRmPropAttribsE()
prop_e = vrm.VRmPropId()
prop_attr_s = vrm.VRmPropAttribsS()
prop_s = ct.c_char_p()

vrm.VRmUsbCamGetPropertyValueF(dev_handle, vrm.VRM_PROPID_CAM_EXPOSURE_TIME_F, ct.byref(prop_f))
prop_exposure_time = prop_f.value
print("Previous exposure time: {:.1f} ms".format(prop_exposure_time))
prop_exposure_time = (prop_exposure_time + 27) % 20
prop_f.value = prop_exposure_time
vrm.VRmUsbCamSetPropertyValueF(dev_handle, vrm.VRM_PROPID_CAM_EXPOSURE_TIME_F, ct.byref(prop_f))
vrm.VRmUsbCamGetPropertyValueF(dev_handle, vrm.VRM_PROPID_CAM_EXPOSURE_TIME_F, ct.byref(prop_f))
prop_exposure_time = prop_f.value
print("New exposure time: {:.1f} ms".format(prop_exposure_time))

####################

vrm.VRmUsbCamStart(dev_handle)
frames_dropped = vrm.VRmBOOL()
img_ready = vrm.VRmBOOL()
img = vrm.POINTER(vrm.VRmImage)()
for ii in range(1):
    vrm.VRmUsbCamLockNextImage(dev_handle, ct.byref(img), ct.byref(frames_dropped))
    height = img.contents.m_image_format.m_height
    width = img.contents.m_image_format.m_width
    pitch = img.contents.m_pitch
    print('pitch-width %s' % (pitch-width))
    im = np.array(img.contents.mp_buffer[0:height * pitch]).reshape(height, pitch)
    print("hxwxp: {:d}x{:d}x{:d}".format(height, width, pitch))
    
    vrm.VRmUsbCamUnlockNextImage(dev_handle, ct.byref(img))
r = vrm.VRmUsbCamStop(dev_handle)

####################

vrm.VRmUsbCamCloseDevice(dev_handle)
vrm.VRmUsbCamFreeDeviceKey(dev_key)


plt.imshow(im)